In [34]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/abalone.csv")

print(df.head())

   id Sex  Length  Diameter  Height  Whole_weight  Shucked_weight  \
0   0   M   0.455     0.365   0.095        0.5140          0.2245   
1   1   M   0.350     0.265   0.090        0.2255          0.0995   
2   2   F   0.530     0.420   0.135        0.6770          0.2565   
3   3   M   0.440     0.365   0.125        0.5160          0.2155   
4   4   I   0.330     0.255   0.080        0.2050          0.0895   

   Viscera_weight  Shell_weight  Rings  
0          0.1010         0.150     15  
1          0.0485         0.070      7  
2          0.1415         0.210      9  
3          0.1140         0.155     10  
4          0.0395         0.055      7  


In [35]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras import layers, models

In [36]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [37]:
label_encoder = LabelEncoder()
df['Sex'] = label_encoder.fit_transform(df['Sex'])

In [38]:
X = df.drop('Rings', axis=1)
y = df['Rings']

In [39]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [40]:
num_features = X_scaled.shape[1]

In [41]:
X_scaled = X_scaled.reshape(X_scaled.shape[0], num_features, 1, 1)

In [42]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [43]:
print(X_train.shape, X_test.shape)

(3341, 9, 1, 1) (836, 9, 1, 1)


In [44]:
model = models.Sequential([
    layers.Conv2D(32, (1, 1), activation='relu', input_shape=(3, 3, 1), padding='same'),
    layers.MaxPooling2D((1, 1)),
    layers.Conv2D(64, (1, 1), activation='relu', padding='same'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='linear')
])


model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['mae'])

history = model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test))


Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


105/105 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 47.4802 - mae: 5.6312 - val_loss: 6.5234 - val_mae: 1.7755
Epoch 2/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 6.3352 - mae: 1.7961 - val_loss: 5.6208 - val_mae: 1.8120
Epoch 3/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 5.3296 - mae: 1.6986 - val_loss: 5.1997 - val_mae: 1.7181
Epoch 4/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 4.9722 - mae: 1.6060 - val_loss: 4.9742 - val_mae: 1.6807
Epoch 5/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 4.8387 - mae: 1.5784 - val_loss: 4.8782 - val_mae: 1.6843
Epoch 6/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 4.6316 - mae: 1.5736 - val_loss: 4.8434 - val_mae: 1.5607
Epoch 7/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 4.8060 - mae: 1.5897 - val_loss: 4.7649 - val_mae: 1.6074
Epoch 8/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 4.8386 - mae: 1.5963 - val_loss: 5.1147 - val_mae: 1.7639
Epoch 9/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 4.7595

In [45]:
model.save('/content/drive/MyDrive/cnn_abalone_model.h5')

In [46]:
from tensorflow.keras.models import load_model

pretrained_model = load_model('/content/drive/MyDrive/cnn_abalone_model.h5')

In [47]:
for layer in pretrained_model.layers[:-2]:
    layer.trainable = False

In [48]:
from tensorflow.keras import layers, models

new_model = models.Sequential(pretrained_model.layers)
new_model.add(layers.Dense(128, activation='relu'))
new_model.add(layers.Dense(1, activation='linear'))


new_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
                  loss='mean_squared_error',
                  metrics=['mae'])


In [49]:
history_finetune = new_model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test))

Epoch 1/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 101.5927 - mae: 9.5936 - val_loss: 97.9837 - val_mae: 9.3708
Epoch 2/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 98.6832 - mae: 9.4091 - val_loss: 93.0788 - val_mae: 9.1185
Epoch 3/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 91.3871 - mae: 9.0610 - val_loss: 87.6321 - val_mae: 8.8283
Epoch 4/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 86.7306 - mae: 8.8282 - val_loss: 81.6813 - val_mae: 8.4986
Epoch 5/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 81.2045 - mae: 8.4775 - val_loss: 75.3388 - val_mae: 8.1313
Epoch 6/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 73.2977 - mae: 8.0590 - val_loss: 68.6838 - val_mae: 7.7259
Epoch 7/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 66.9974 - mae: 7.6637 - val_loss: 61.8905 - val_mae: 7.2877
Epoch 8/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 61.2801 - mae: 7.2887 - val_loss: 55.0653 - val_mae: 6.8182
Epoch 9/20
105/105 ━━━━━━━━━━━━━━━━━━━━